### SECTION 1: ENVIRONMENT SETUP

In [75]:
# Install necessary libraries
!pip install pymysql
!pip install mysql-connector-python
!pip install sqlalchemy
!pip install pymongo
!pip install pandas
!pip install matplotlib

In [76]:
# Import libraries
import os
import pymysql
import mysql.connector
from sqlalchemy import create_engine
import pymongo
import datetime
import json
import requests

import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

### SECTION 2: CONNECT TO THE ORDER_MANAGEMENT DATABASE

In [77]:
# Database connection configuration
host = "localhost"
user = "root"
password = "ds2002spring2024"
main_database = "order_management"
extract_database = "northwind"

# Create an SQLAlchemy engine for the order_management database
order_management_engine = create_engine(f'mysql+pymysql://{user}:{password}@{host}/{main_database}')

# Connect to the order_management database
try:
    order_management_connection = pymysql.connect(host=host, user=user, password=password, database=main_database)
    print("Connection to the order_management database was successful!")
except Exception as e:
    print(f"Error connecting to the order_management database: {e}")

Connection to the order_management database was successful!


### SECTION 3: EXTRACT DATA FROM THE NORTHWIND MYSQL DATABASE AND LOAD DATA INTO THE ORDER_MANAGEMENT DATABASE

In [78]:
# Create an SQLAlchemy engine for the northwind database
northwind_engine = create_engine(f'mysql+pymysql://{user}:{password}@{host}/{extract_database}')

# Connect to the northwind database
try:
    northwind_connection = pymysql.connect(host=host, user=user, password=password, database=extract_database)
    print("Connection to the northwind database was successful!")
except Exception as e:
    print(f"Error connecting to the northwind database: {e}")

Connection to the northwind database was successful!


In [83]:
# Extract customers data from the northwind database
try:
    northwind_customers_query = "SELECT id, first_name, last_name, email_address, business_phone, address FROM customers"
    northwind_customers_df = pd.read_sql(northwind_customers_query, northwind_connection)
except Exception as e:
    print(f"Error extracting data from the Northwind database: {e}")

# Modify the columns from the northwind database to fit the order_management database
try:
    northwind_customers_df.rename(columns={
        'id': 'CustomerID',
        'first_name': 'FirstName',
        'last_name': 'LastName',
        'email_address': 'Email',
        'business_phone': 'Phone',
        'address': 'Address'
    }, inplace=True)
except Exception as e:
    print(f"Error modifying data: {e}")

northwind_customers_df.head()

,CustomerID,FirstName,LastName,Email,Phone,Address
0,1,Anna,Bedecs,None,(123)555-0100,123 1st Street
1,2,Antonio,Gratacos Solsona,None,(123)555-0100,123 2nd Street
2,3,Thomas,Axen,None,(123)555-0100,123 3rd Street
3,4,Christina,Lee,None,(123)555-0100,123 4th Street
4,5,Martin,O’Donnell,None,(123)555-0100,123 5th Street


In [79]:
# Load the customers data into the order_management database
try:
    northwind_customers_df.to_sql('Customers', order_management_engine, if_exists='append', index=False)
except Exception as e:
    print(f"Error loading data into the order_management database: {e}")

In [84]:
# Extract products data from the northwind database
try:
    northwind_products_query = "SELECT id, product_name, category, list_price, minimum_reorder_quantity FROM products"
    northwind_products_df = pd.read_sql(northwind_products_query, northwind_connection)
except Exception as e:
    print(f"Error extracting data from the Northwind database: {e}")

# Modify the columns from the northwind database to fit the order_management database
try:
    northwind_products_df.rename(columns={
        'id': 'ProductID',
        'product_name': 'Name',
        'category': 'Description',
        'list_price': 'Price',
        'minimum_reorder_quantity': 'StockQuantity'
    }, inplace=True)
except Exception as e:
    print(f"Error modifying data: {e}")

northwind_products_df.head()

,ProductID,Name,Description,Price,StockQuantity
0,1,Northwind Traders Chai,Beverages,18.00,10.0
1,3,Northwind Traders Syrup,Condiments,10.00,25.0
2,4,Northwind Traders Cajun Seasoning,Condiments,22.00,10.0
3,5,Northwind Traders Olive Oil,Oil,21.35,10.0
4,6,Northwind Traders Boysenberry Spread,"Jams, Preserves",25.00,25.0


In [81]:
# Load the products data into the order_management database
try:
    northwind_products_df.to_sql('Products', order_management_engine, if_exists='append', index=False)
except Exception as e:
    print(f"Error loading data into the order_management database: {e}")

In [82]:
# Extract orders data from the northwind database
try:
    northwind_orders_query = "SELECT id, customer_id, order_date, ship_address, status_id FROM orders"
    northwind_orders_df = pd.read_sql(northwind_orders_query, northwind_connection)
except Exception as e:
    print(f"Error extracting data from the Northwind database: {e}")

# Modify the columns from the northwind database to fit the order_management database
try:
    northwind_orders_df.rename(columns={
        'id': 'OrderID',
        'customer_id': 'CustomerID',
        'order_date': 'OrderDate',
        'ship_address': 'ShippingAddress',
        'status_id': 'Status'
    }, inplace=True)
except Exception as e:
    print(f"Error modifying data: {e}")

northwind_orders_df.head()

,OrderID,CustomerID,OrderDate,ShippingAddress,Status
0,30,27,2006-01-15,789 27th Street,3
1,31,4,2006-01-20,123 4th Street,3
2,32,12,2006-01-22,123 12th Street,3
3,33,8,2006-01-30,123 8th Street,3
4,34,4,2006-02-06,123 4th Street,3


In [85]:
# Load the orders data into the order_management database
try:
    northwind_orders_df.to_sql('Orders', order_management_engine, if_exists='append', index=False)
except Exception as e:
    print(f"Error loading data into the order_management database: {e}")

In [86]:
# Extract order details (items) data from the northwind database
try:
    northwind_order_items_query = "SELECT id, order_id, product_id, quantity, unit_price FROM order_details"
    northwind_order_items_df = pd.read_sql(northwind_order_items_query, northwind_connection)
except Exception as e:
    print(f"Error extracting data from the Northwind database: {e}")

# Modify the columns from the northwind database to fit the order_management database
try:
    northwind_order_items_df.rename(columns={
        'id': 'OrderItemID',
        'order_id': 'OrderID',
        'product_id': 'ProductID',
        'quantity': 'Quantity',
        'unit_price': 'Price'
    }, inplace=True)
except Exception as e:
    print(f"Error modifying data: {e}")

northwind_order_items_df.head()

,OrderItemID,OrderID,ProductID,Quantity,Price
0,27,30,34,100.0,14.0
1,28,30,80,30.0,3.5
2,29,31,7,10.0,30.0
3,30,31,51,10.0,53.0
4,31,31,80,10.0,3.5


In [87]:
# Load the order details (items) data into the order_management database
try:
    northwind_order_items_df.to_sql('OrderItems', order_management_engine, if_exists='append', index=False)
except Exception as e:
    print(f"Error loading data into the order_management database: {e}")

In [88]:
# Extract suppliers data from the northwind database
try:
    northwind_suppliers_query = "SELECT id, first_name, last_name, business_phone, address, company FROM suppliers"
    northwind_suppliers_df = pd.read_sql(northwind_suppliers_query, northwind_connection)
except Exception as e:
    print(f"Error extracting data from the Northwind database: {e}")

# Modify the columns from the northwind database to fit the order_management database
try:
    northwind_suppliers_df.rename(columns={
        'id': 'SupplierID',
        'first_name': 'FirstName',
        'last_name': 'LastName',
        'business_phone': 'Phone',
        'address': 'Address',
        'company': 'Company'
    }, inplace=True)
except Exception as e:
    print(f"Error modifying data: {e}")

northwind_suppliers_df.head()

,SupplierID,FirstName,LastName,Phone,Address,Company
0,1,Elizabeth A.,Andersen,None,None,Supplier A
1,2,Cornelia,Weiler,None,None,Supplier B
2,3,Madeleine,Kelley,None,None,Supplier C
3,4,Naoki,Sato,None,None,Supplier D
4,5,Amaya,Hernandez-Echevarria,None,None,Supplier E


In [89]:
# Load the suppliers data into the order_management database
try:
    northwind_suppliers_df.to_sql('Suppliers', order_management_engine, if_exists='append', index=False)
except Exception as e:
    print(f"Error loading data into the order_management database: {e}")

### SECTION 4: EXTRACT DATA FROM A CSV FILE AND LOAD DATA INTO THE ORDER_MANAGEMENT DATABASE

In [90]:
# Load the CSV file into a DataFrame
csv_sales_data_df = pd.read_csv('sales_data_sample.csv', encoding='ISO-8859-1')
csv_sales_data_df.head()

,ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,...,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE
0,10107,30,95.70,2,2871.00,2/24/2003 0:00,Shipped,1,2,2003,...,897 Long Airport Avenue,NaN,NYC,NY,10022,USA,NaN,Yu,Kwai,Small
1,10121,34,81.35,5,2765.90,5/7/2003 0:00,Shipped,2,5,2003,...,59 rue de l'Abbaye,NaN,Reims,NaN,51100,France,EMEA,Henriot,Paul,Small
2,10134,41,94.74,2,3884.34,7/1/2003 0:00,Shipped,3,7,2003,...,27 rue du Colonel Pierre Avia,NaN,Paris,NaN,75508,France,EMEA,Da Cunha,Daniel,Medium
3,10145,45,83.26,6,3746.70,8/25/2003 0:00,Shipped,3,8,2003,...,78934 Hillside Dr.,NaN,Pasadena,CA,90003,USA,NaN,Young,Julie,Medium
4,10159,49,100.00,14,5205.27,10/10/2003 0:00,Shipped,4,10,2003,...,7734 Strong St.,NaN,San Francisco,CA,NaN,USA,NaN,Brown,Julie,Medium


In [54]:
# Extract and transform customers data from the CSV file
try:
    csv_customers_df = csv_sales_data_df[['CUSTOMERNAME', 'PHONE', 'ADDRESSLINE1']].copy()
    csv_customers_df.drop_duplicates(inplace=True)
    # Modify the columns from the CSV file to fit the order_management database
    csv_customers_df.rename(columns={
        'CUSTOMERNAME': 'FirstName',
        'PHONE': 'Phone',
        'ADDRESSLINE1': 'Address'
    }, inplace=True)
    csv_customers_df.insert(1, 'LastName', '')
    csv_customers_df.insert(2, 'Email', '')
except Exception as e:
    print(f"Error extracting and transforming customer data: {e}")

csv_customers_df.head()

,FirstName,LastName,Email,Phone,Address
0,Land of Toys Inc.,,,2125557818,897 Long Airport Avenue
1,Reims Collectables,,,26.47.1555,59 rue de l'Abbaye
2,Lyon Souveniers,,,+33 1 46 62 7555,27 rue du Colonel Pierre Avia
3,Toys4GrownUps.com,,,6265557265,78934 Hillside Dr.
4,Corporate Gift Ideas Co.,,,6505551386,7734 Strong St.


In [91]:
# Load the customers data into the order_management database
try:
    csv_customers_df.to_sql('Customers', order_management_engine, if_exists='append', index=False)
except Exception as e:
    print(f"Error loading data into the order_management database: {e}")

In [92]:
# Extract and transform products data from the CSV file
try:
    csv_products_df = csv_sales_data_df[['PRODUCTLINE', 'MSRP', 'QUANTITYORDERED']].copy()
    csv_products_df.drop_duplicates(inplace=True)
    # Modify the columns from the CSV file to fit the order_management database
    csv_products_df.rename(columns={
        'PRODUCTLINE': 'Name',
        'MSRP': 'Price',
        'QUANTITYORDERED': 'StockQuantity'
    }, inplace=True)
    csv_products_df.insert(1, 'Description', '')
except Exception as e:
    print(f"Error extracting and transforming product data: {e}")

csv_products_df.head()

,Name,Description,Price,StockQuantity
0,Motorcycles,,95,30
1,Motorcycles,,95,34
2,Motorcycles,,95,41
3,Motorcycles,,95,45
4,Motorcycles,,95,49


In [93]:
# Load the products data into the order_management database
try:
    csv_products_df.to_sql('Products', order_management_engine, if_exists='append', index=False)
except Exception as e:
    print(f"Error loading data into the order_management database: {e}")

In [96]:
# Extract and transform orders data from the CSV file
try:
    csv_orders_df = csv_sales_data_df[['ORDERNUMBER', 'ORDERDATE', 'STATUS']].copy()
    csv_orders_df.drop_duplicates(inplace=True)
    # Modify the columns from the CSV file to fit the order_management database
    csv_orders_df.rename(columns={
        'ORDERNUMBER': 'OrderID',
        'ORDERDATE': 'OrderDate',
        'STATUS': 'Status'
    }, inplace=True)
    csv_orders_df.insert(3, 'ShippingAddress', '')
    # Convert the 'OrderDate' column to datetime format
    csv_orders_df['OrderDate'] = pd.to_datetime(csv_orders_df['OrderDate'], format='%m/%d/%Y %H:%M')
except Exception as e:
    print(f"Error extracting and transforming order data: {e}")

csv_orders_df.head()

,OrderID,OrderDate,Status,ShippingAddress
0,10107,2003-02-24,Shipped,
1,10121,2003-05-07,Shipped,
2,10134,2003-07-01,Shipped,
3,10145,2003-08-25,Shipped,
4,10159,2003-10-10,Shipped,


In [97]:
# Load the orders data into the order_management database
try:
    csv_orders_df.to_sql('Orders', order_management_engine, if_exists='append', index=False)
except Exception as e:
    print(f"Error loading data into the order_management database: {e}")

In [98]:
# Extract and transform order details (items) data from the CSV file
try:
    csv_order_items_df = csv_sales_data_df[['ORDERNUMBER', 'QUANTITYORDERED', 'PRICEEACH']].copy()
    # Modify the columns from the CSV file to fit the order_management database
    csv_order_items_df.rename(columns={
        'ORDERNUMBER': 'OrderID',
        'QUANTITYORDERED': 'Quantity',
        'PRICEEACH': 'Price'
    }, inplace=True)
except Exception as e:
    print(f"Error extracting and transforming order items data: {e}")

csv_order_items_df.head()

,OrderID,Quantity,Price
0,10107,30,95.70
1,10121,34,81.35
2,10134,41,94.74
3,10145,45,83.26
4,10159,49,100.00


In [99]:
# Load the order details (items) data into the order_management database
try:
    csv_order_items_df.to_sql('OrderItems', order_management_engine, if_exists='append', index=False)
except Exception as e:
    print(f"Error loading data into the order_management database: {e}")

In [100]:
# Extract and transform suppliers data from the CSV file
try:
    csv_suppliers_df = csv_sales_data_df[['CONTACTFIRSTNAME', 'CONTACTLASTNAME', 'PHONE', 'ADDRESSLINE1']].copy()
    csv_suppliers_df.drop_duplicates(inplace=True)
    # Modify the columns from the CSV file to fit the order_management database
    csv_suppliers_df.rename(columns={
        'CONTACTFIRSTNAME': 'FirstName',
        'CONTACTLASTNAME': 'LastName',
        'PHONE': 'Phone',
        'ADDRESSLINE1': 'Address'
    }, inplace=True)
    csv_suppliers_df.insert(4, 'Company', '')
except Exception as e:
    print(f"Error extracting and transforming supplier data: {e}")

csv_suppliers_df.head()

,FirstName,LastName,Phone,Address,Company
0,Kwai,Yu,2125557818,897 Long Airport Avenue,
1,Paul,Henriot,26.47.1555,59 rue de l'Abbaye,
2,Daniel,Da Cunha,+33 1 46 62 7555,27 rue du Colonel Pierre Avia,
3,Julie,Young,6265557265,78934 Hillside Dr.,
4,Julie,Brown,6505551386,7734 Strong St.,


In [101]:
# Load the suppliers data into the order_management database
try:
    csv_suppliers_df.to_sql('Suppliers', order_management_engine, if_exists='append', index=False)
except Exception as e:
    print(f"Error loading data into the order_management database: {e}")

### SECTION 5: EXTRACT DATA FROM AN API CALL AND LOAD DATA INTO THE ORDER_MANAGEMENT DATABASE

In [102]:
# Define the URL for the Fake Store API's products endpoint
api_url = 'https://fakestoreapi.com/products'

# Send a GET request to the API to retrieve product data
response = requests.get(api_url)

In [74]:
# Check if the request was successful
if response.status_code == 200:
    try:
        # Convert the JSON response to a DataFrame
        api_products_data = response.json()
        api_products_df = pd.DataFrame(api_products_data)
        # Extract and transform products data from the API call
        api_products_df = api_products_df[['title', 'description', 'price']]
        # Modify the columns from the API call to fit the order_management database
        api_products_df.rename(columns={
            'title': 'Name',
            'description': 'Description',
            'price': 'Price',
        }, inplace=True)
    except Exception as e:
        print(f"Error converting API response to DataFrame: {e}")

api_products_df.head()

,Name,Description,Price
0,"Fjallraven - Foldsack No. 1 Backpack, Fits 15 ...",Your perfect pack for everyday use and walks i...,109.95
1,Mens Casual Premium Slim Fit T-Shirts,"Slim-fitting style, contrast raglan long sleev...",22.30
2,Mens Cotton Jacket,great outerwear jackets for Spring/Autumn/Wint...,55.99
3,Mens Casual Slim Fit,The color could be slightly different between ...,15.99
4,John Hardy Women's Legends Naga Gold & Silver ...,"From our Legends Collection, the Naga was insp...",695.00


In [103]:
# Load the products data into the order_management database
try:
    api_products_df.to_sql('Products', order_management_engine, if_exists='append', index=False)
except Exception as e:
    print(f"Error loading data into the order_management database: {e}")

### SECTION 6: AUTHOR SQL QUERIES (SELECT STATEMENTS) TO DEMONSTRATE PROPER DATABASE FUNCTIONALITY

In [104]:
# Connect to the order_management database
try:
    sql_queries_connection = pymysql.connect(host=host, user=user, password=password, database=main_database)
    print("Connection to the order_management database was successful!")
except Exception as e:
    print(f"Error connecting to the order_management database: {e}")

Connection to the order_management database was successful!


In [105]:
# Query 1: Retrieve Customer Information with Total Order Amounts
df_customer_sales = pd.read_sql("""
    SELECT
        c.CustomerID,
        c.FirstName,
        c.LastName,
        SUM(od.Quantity * p.Price) AS TotalAmount
    FROM
        Customers c
        JOIN Orders o ON c.CustomerID = o.CustomerID
        JOIN OrderItems od ON o.OrderID = od.OrderID
        JOIN Products p ON od.ProductID = p.ProductID
    GROUP BY
        c.CustomerID,
        c.FirstName,
        c.LastName
    ORDER BY
        TotalAmount DESC;
    """, sql_queries_connection)

print("Query 1: Customer Information with Total Order Amounts")
df_customer_sales.head()

Query 1: Customer Information with Total Order Amounts


,CustomerID,FirstName,LastName,TotalAmount
0,28,Amritansh,Raghav,15432.5
1,7,Ming-Yang,Xie,13800.0
2,6,Francisco,Pérez-Olaeta,8007.5
3,4,Christina,Lee,4949.0
4,8,Elizabeth,Andersen,4683.0


In [106]:
# Query 2: Calculate Average Order Amount by Customer
df_average_order_amount = pd.read_sql("""
    SELECT
        c.CustomerID,
        c.FirstName,
        c.LastName,
        AVG(od.Quantity * p.Price) AS AverageOrderAmount
    FROM
        Customers c
        JOIN Orders o ON c.CustomerID = o.CustomerID
        JOIN OrderItems od ON o.OrderID = od.OrderID
        JOIN Products p ON od.ProductID = p.ProductID
    GROUP BY
        c.CustomerID,
        c.FirstName,
        c.LastName
    ORDER BY
        AverageOrderAmount DESC;
    """, sql_queries_connection)

print("\nQuery 2: Average Order Amount by Customer")
df_average_order_amount.head()


Query 2: Average Order Amount by Customer


,CustomerID,FirstName,LastName,AverageOrderAmount
0,7,Ming-Yang,Xie,13800.000000
1,28,Amritansh,Raghav,3858.125000
2,6,Francisco,Pérez-Olaeta,1334.583333
3,9,Sven,Mortensen,1262.166667
4,29,Soo Jung,Lee,968.500000


In [107]:
# Query 3: Retrieve Product Information with Total Sales
df_product_sales = pd.read_sql("""
    SELECT
        p.ProductID,
        p.Name AS ProductName,
        SUM(od.Quantity) AS TotalUnitsSold,
        SUM(od.Quantity * p.Price) AS TotalSalesAmount
    FROM
        Products p
        JOIN OrderItems od ON p.ProductID = od.ProductID
    GROUP BY
        p.ProductID,
        p.Name
    ORDER BY
        TotalSalesAmount DESC;
    """, sql_queries_connection)

print("\nQuery 3: Product Information with Total Sales")
df_product_sales.head()


Query 3: Product Information with Total Sales


,ProductID,ProductName,TotalUnitsSold,TotalSalesAmount
0,43,Northwind Traders Coffee,650.0,29900.0
1,34,Northwind Traders Beer,487.0,6818.0
2,20,Northwind Traders Marmalade,40.0,3240.0
3,72,Northwind Traders Mozzarella,90.0,3132.0
4,41,Northwind Traders Clam Chowder,290.0,2798.5
